# Pydantic fundamental

## Start without pydantic

In [1]:
class Person:
    def __init__(self, name:str, gender:str, age:int) -> None:
        self.name = name
        self.gender = gender
        self.age = age


person1 = Person(name= "Marcus", age= 33, gender= "M")
person1

In [3]:
person1.name, person1.age

('Marcus', 33)

In [7]:
person2 = Person(name= 321+321,gender=True, age= "minus ten")
person2



## Validation of out person class

In [39]:
class Person:
    def __init__(self, name:str, gender:str, age:int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name
        self.gender = gender
        
        if not isinstance(age, int):
            raise TypeError(f"age must be of {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124 not {age}")
        self.age = age
        

    
    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"
        
        
try:        
    person3 = Person(name= "Ragnar",gender="M", age= -54)
except ValueError as err:
    print(err)
    
person3

age must be between 0 and 124 not -54


Person(Ragnar, M, -54)

In [21]:
person4 = Person(name= "Bella",gender="F", age= 4)
person4

Person(Bella, F, 4)

In [23]:
person4.age = -5

person4

Person(Bella, F, -5)

In [ ]:
class Person:
    def __init__(self, name:str, gender:str, age:int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"name must be of type str not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age
        
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, age):
        if not isinstance(age, int):
            raise TypeError(f"age must be of {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124 not {age}")
        self._age = age
        
        
    def __repr__(self):
        return f"Person({self.name}, {self.gender}, {self.age})"
    

person5 = Person(name= "Nataly",gender="F", age= 4)


In [ ]:

try:
    
    person5.age = 10
except ValueError as err:
    print(err)




In [40]:

try:
    Person(name= "Nataly",gender="F", age= "4")
except TypeError as err:
    print(err)


age must be of <class 'str'>


## Validate using Pydantic

In [41]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    gender: str
    age: int 
    
person6 = Person(name="Christina", gender= "F", age= 29)
person6

Person(name='Christina', gender='F', age=29)

In [43]:
person6.age = 36
person6

Person(name='Christina', gender='F', age=36)

In [ ]:
# issue
person6.age = "thirty"
person6

Person(name='Christina', gender='F', age='thirty')

In [47]:
from pydantic import ValidationError

try:
    Person(name=13242, gender= "F", age= 29)
except ValidationError as err:
    print(err)


1 validation error for Person
name
  Input should be a valid string [type=string_type, input_value=13242, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [ ]:
# str "29" is coerced into 29
Person(name="Nina", gender= "F", age= "29")

Person(name='Nina', gender='F', age=29)

In [53]:
from pydantic import ConfigDict

class Person(BaseModel):
    name: str
    gender: str
    age: int 
    
    model_config = ConfigDict(validate_assignment= True)
  
  
person7 = Person(name= "Nina", gender= "F", age= "29")
    
try:
    person7.age = "thirty"
except ValidationError as err:
    print(err)  
    
person7.age = 10
person7

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


Person(name='Nina', gender='F', age=10)

In [54]:

try:
    Person(name= 1.342, gender= 3.452, age= -4)
except ValidationError as err:
    print(err) 




2 validation errors for Person
name
  Input should be a valid string [type=string_type, input_value=1.342, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=3.452, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [ ]:

try:
    Person(name= "Sofia", gender= "F", age= -4)
except ValidationError as err:
    print(err) 
    


## Add age validation

In [57]:
from pydantic import Field
from typing import Literal

class Person(BaseModel):
    name: str
    gender: Literal["M", "F"]
    age: int = Field(gt = -1, lt = 125)
    
    model_config = ConfigDict(validate_assignment= True)
    
    
    

try:
    Person(name= "Sofia", gender= "Fem", age= -4)
except ValidationError as err:
    print(err)

2 validation errors for Person
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='Fem', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


## Serialization and deserialization

In [ ]:


# serialization 
person7.model_dump()



{'name': 'Nina', 'gender': 'F', 'age': 10}

In [ ]:
import json

with open("person.json", "w") as file:
    json.dump(person7.model_dump(), file)




In [63]:
with open("person.json", "r") as file:
    json_data = file.read()
    

print(repr(json_data))


# deserialization
Person.model_validate_json(json_data)



'{"name": "Nina", "gender": "F", "age": 10}'


Person(name='Nina', gender='F', age=10)